In [ ]:
import h5py
import numpy as np
import k3d
import trimesh
import os
from omegaconf import OmegaConf
from airexo.helpers.urdf_robot import forward_kinematic
from airexo.helpers.constants import ROBOT_PREDEFINED_TRANSFORMATION

# 1. 读取关节数据
scene_path = "/data/haoxiang/data/airexo2/task_0013/train/scene_0001/lowdim"

with h5py.File(f"{scene_path}/robot_left.h5", 'r') as f:
    left_joint = f['joint_pos'][0]  # 第一帧，shape=(7,)

with h5py.File(f"{scene_path}/robot_right.h5", 'r') as f:
    right_joint = f['joint_pos'][0]

with h5py.File(f"{scene_path}/gripper_left.h5", 'r') as f:
    left_gripper = f['width'][0] if 'width' in f else 0.05

with h5py.File(f"{scene_path}/gripper_right.h5", 'r') as f:
    right_gripper = f['width'][0] if 'width' in f else 0.05

# 拼接关节+夹爪
left_joint = np.concatenate([left_joint, [left_gripper]])
right_joint = np.concatenate([right_joint, [right_gripper]])

# 2. 加载 joint_cfgs
left_cfg = OmegaConf.load("airexo/configs/joint/left/robot.yaml")
right_cfg = OmegaConf.load("airexo/configs/joint/right/robot.yaml")

# 3. 前向运动学
cur_transforms, visuals_map = forward_kinematic(
    left_joint=left_joint,
    right_joint=right_joint,
    left_joint_cfgs=left_cfg,
    right_joint_cfgs=right_cfg,
    is_rad=True, 
    urdf_file="airexo/urdf_models/robot/robot_inhand.urdf",
    with_visuals_map=True
)

# 4. 加载并变换 mesh
plot = k3d.plot()
for link, transform in cur_transforms.items():
    if link not in visuals_map: continue
    
    for v in visuals_map[link]:
        if v.geom_param is None: continue
        
        # 构建完整路径
        mesh_path = os.path.join("airexo/urdf_models/robot", v.geom_param)
        
        if not os.path.exists(mesh_path):
            print(f"Warning: Mesh file not found: {mesh_path}")
            continue

        # 使用 force='mesh' 确保加载进来的是 Trimesh 对象而不是 Scene
        mesh = trimesh.load(mesh_path, force='mesh')
        
        # 应用变换矩阵
        tf = ROBOT_PREDEFINED_TRANSFORMATION @ transform.matrix() @ v.offset.matrix()
        mesh.apply_transform(tf)
        
        # 渲染到 k3d
        plot += k3d.mesh(mesh.vertices.astype(np.float32), 
                         mesh.faces.astype(np.uint32),
                         color=0xd3d3d3) # 浅灰色

plot.display()

In [ ]:
import h5py
import numpy as np
import k3d
import trimesh
import os
from omegaconf import OmegaConf
from airexo.helpers.urdf_robot import forward_kinematic
from airexo.helpers.constants import ROBOT_PREDEFINED_TRANSFORMATION,O3D_RENDER_TRANSFORMATION
from airexo.calibration.calib_info import CalibrationInfo
import open3d as o3d

# ========== 1. 读取关节数据 ==========
scene_path = "/data/haoxiang/data/airexo2/task_0013/train/scene_0001"
lowdim_path = os.path.join(scene_path, "lowdim")

with h5py.File(f"{lowdim_path}/robot_left.h5", 'r') as f:
    left_joint = f['joint_pos'][0]

with h5py.File(f"{lowdim_path}/robot_right.h5", 'r') as f:
    right_joint = f['joint_pos'][0]

with h5py.File(f"{lowdim_path}/gripper_left.h5", 'r') as f:
    left_gripper = f['width'][0] if 'width' in f else 0.05

with h5py.File(f"{lowdim_path}/gripper_right.h5", 'r') as f:
    right_gripper = f['width'][0] if 'width' in f else 0.05

left_joint = np.concatenate([left_joint, [left_gripper]])
right_joint = np.concatenate([right_joint, [right_gripper]])

# ========== 2. 加载标定数据 ==========
calib_path = "/data/haoxiang/data/airexo2/task_0013/calib"
calib_info = CalibrationInfo(
    calib_path=calib_path,
    calib_timestamp=1737548651048  # 自动找最新的标定文件
)

# 获取相机序列号
camera_serial = list(calib_info.camera_serials_global)[0]
print(f"Using camera: {camera_serial}")

# 获取相机到机器人基座的变换和内参
cam_to_base = calib_info.get_camera_to_base(camera_serial)
intrinsic = calib_info.get_intrinsic(camera_serial)

# ========== 3. 前向运动学（重建机械臂） ==========
left_cfg = OmegaConf.load("airexo/configs/joint/left/robot.yaml")
right_cfg = OmegaConf.load("airexo/configs/joint/right/robot.yaml")

cur_transforms, visuals_map = forward_kinematic(
    left_joint=left_joint,
    right_joint=right_joint,
    left_joint_cfgs=left_cfg,
    right_joint_cfgs=right_cfg,
    is_rad=True,
    urdf_file="airexo/urdf_models/robot/robot_inhand.urdf",  # 包含中间框架
    with_visuals_map=True
)

# ========== 4. 读取深度图并转换为点云 ==========
cam_path = os.path.join(scene_path, f"cam_{camera_serial}")
print(f"Reading depth and RGB from: {cam_path}")
depth_path = os.path.join(cam_path, "depth", "1737546126606.png")  # 第一帧
rgb_path = os.path.join(cam_path, "color", "1737546126606.png")

# 读取深度和RGB
depth_img = o3d.io.read_image(depth_path)
rgb_img = o3d.io.read_image(rgb_path)

# 创建 RGBD 图像
rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
    rgb_img, depth_img,
    depth_scale=1000.0,  # 深度单位转米
    convert_rgb_to_intensity=False
)

print(f"Intrinsic type: {type(intrinsic)}")
print(f"Intrinsic content:\n{intrinsic}")

# 转换为点云（相机坐标系）

# 1. 从矩阵中提取 4 个核心参数
fx = intrinsic[0, 0]
fy = intrinsic[1, 1]
cx = intrinsic[0, 2]
cy = intrinsic[1, 2]

# 2. 从 rgb_img 获取分辨率
# 如果 rgb_img 是 Open3D 对象，可以用 np.asarray(rgb_img).shape
h, w = np.asarray(rgb_img).shape[:2] 

# 3. 构造 Open3D 的内参对象
intrinsic_o3d = o3d.camera.PinholeCameraIntrinsic(
    width=int(w),
    height=int(h),
    fx=float(fx),
    fy=float(fy),
    cx=float(cx),
    cy=float(cy)
)

# 4. 转换为点云
pcd_cam = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd,
    intrinsic_o3d
)

# 变换到机器人基座坐标系
pcd_cam.transform(cam_to_base)
# ✅ 正确变换顺序（参照 visualizer.py L969）
# pcd_cam.transform(O3D_RENDER_TRANSFORMATION)  # 只需这一个变换！

# ========== 5. K3D 可视化 ==========
plot = k3d.plot()

# 5.1 渲染机械臂（包括中间框架）
for link, transform in cur_transforms.items():
    if link not in visuals_map: continue
    
    for v in visuals_map[link]:
        if v.geom_param is None: continue
        
        mesh_path = os.path.join("airexo/urdf_models/robot", v.geom_param)
        if not os.path.exists(mesh_path):
            continue

        mesh = trimesh.load(mesh_path, force='mesh')
        tf = ROBOT_PREDEFINED_TRANSFORMATION @ transform.matrix() @ v.offset.matrix()
        mesh.apply_transform(tf)
        
        # 机械臂用灰色
        plot += k3d.mesh(mesh.vertices.astype(np.float32), 
                         mesh.faces.astype(np.uint32),
                         color=0xaaaaaa)

# 5.2 渲染点云
pcd_points = np.asarray(pcd_cam.points).astype(np.float32)

# ✅ 点云也要应用 ROBOT_PREDEFINED_TRANSFORMATION 才能和机械臂对齐
pcd_points_transformed = (ROBOT_PREDEFINED_TRANSFORMATION[:3, :3] @ pcd_points.T + 
                          ROBOT_PREDEFINED_TRANSFORMATION[:3, 3:4]).T


pcd_colors = np.asarray(pcd_cam.colors).astype(np.float32)

# 1. 先映射到 0-255，然后务必转换为 uint32
pcd_colors_uint32 = (pcd_colors * 255).astype(np.uint32)

# 2. 进行位移操作 (R << 16 | G << 8 | B)
pcd_colors_packed = (pcd_colors_uint32[:, 0] << 16) | \
                    (pcd_colors_uint32[:, 1] << 8) | \
                     pcd_colors_uint32[:, 2]

plot += k3d.points(pcd_points, 
                   colors=pcd_colors_packed,
                   point_size=0.002,
                   shader='flat')

# 5.3 添加坐标轴（机器人基座）
axis_size = 0.3
axes = k3d.vectors(
    origins=[[0, 0, 0], [0, 0, 0], [0, 0, 0]],
    vectors=[[axis_size, 0, 0], [0, axis_size, 0], [0, 0, axis_size]],
    colors=[0xff0000, 0x00ff00, 0x0000ff],  # RGB
    line_width=0.01
)
plot += axes

plot.display()

In [21]:
import h5py
import numpy as np
import k3d
import trimesh
import os
from omegaconf import OmegaConf
from airexo.helpers.urdf_robot import forward_kinematic
from airexo.helpers.constants import ROBOT_PREDEFINED_TRANSFORMATION
from airexo.calibration.calib_info import CalibrationInfo
import open3d as o3d

# ========== 1. 读取关节数据 ==========
scene_path = "/data/haoxiang/data/airexo2/task_0013/train/scene_0001"
lowdim_path = os.path.join(scene_path, "lowdim")

with h5py.File(f"{lowdim_path}/robot_left.h5", 'r') as f:
    left_joint = f['joint_pos'][0]

with h5py.File(f"{lowdim_path}/robot_right.h5", 'r') as f:
    right_joint = f['joint_pos'][0]

with h5py.File(f"{lowdim_path}/gripper_left.h5", 'r') as f:
    left_gripper = f['width'][0] if 'width' in f else 0.05

with h5py.File(f"{lowdim_path}/gripper_right.h5", 'r') as f:
    right_gripper = f['width'][0] if 'width' in f else 0.05

left_joint = np.concatenate([left_joint, [left_gripper]])
right_joint = np.concatenate([right_joint, [right_gripper]])

# ========== 2. 加载标定数据 ==========
calib_path = "/data/haoxiang/data/airexo2/task_0013/calib"
calib_info = CalibrationInfo(
    calib_path=calib_path,
    calib_timestamp=1737548651048
)

camera_serial = list(calib_info.camera_serials_global)[0]
print(f"Using camera: {camera_serial}")

cam_to_base = calib_info.get_camera_to_base(camera_serial)
intrinsic = calib_info.get_intrinsic(camera_serial)

# ========== DEBUG: 打印标定信息 ==========
print("\n=== DEBUG: Calibration Info ===")
print(f"Camera to base transformation:\n{cam_to_base}")
print(f"\nROBOT_PREDEFINED_TRANSFORMATION:\n{ROBOT_PREDEFINED_TRANSFORMATION}")
print(f"\nIntrinsic matrix:\n{intrinsic}")

# ========== 3. 前向运动学 ==========
left_cfg = OmegaConf.load("airexo/configs/joint/left/robot.yaml")
right_cfg = OmegaConf.load("airexo/configs/joint/right/robot.yaml")

cur_transforms, visuals_map = forward_kinematic(
    left_joint=left_joint,
    right_joint=right_joint,
    left_joint_cfgs=left_cfg,
    right_joint_cfgs=right_cfg,
    is_rad=True,
    urdf_file="airexo/urdf_models/robot/robot_inhand.urdf",
    with_visuals_map=True
)

# ========== DEBUG: 打印机械臂变换 ==========
print("\n=== DEBUG: Robot Transform (first link) ===")
first_link = list(cur_transforms.keys())[0]
print(f"First link: {first_link}")
print(f"Transform matrix:\n{cur_transforms[first_link].matrix()}")

# ========== 4. 读取深度图并转换为点云 ==========
cam_path = os.path.join(scene_path, f"cam_{camera_serial}")
depth_path = os.path.join(cam_path, "depth", "1737546126606.png")
rgb_path = os.path.join(cam_path, "color", "1737546126606.png")

depth_img = o3d.io.read_image(depth_path)
rgb_img = o3d.io.read_image(rgb_path)

rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
    rgb_img, depth_img,
    depth_scale=1000.0,
    convert_rgb_to_intensity=False
)

fx = intrinsic[0, 0]
fy = intrinsic[1, 1]
cx = intrinsic[0, 2]
cy = intrinsic[1, 2]
h, w = np.asarray(rgb_img).shape[:2]

intrinsic_o3d = o3d.camera.PinholeCameraIntrinsic(
    width=int(w), height=int(h),
    fx=float(fx), fy=float(fy),
    cx=float(cx), cy=float(cy)
)

pcd_cam = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic_o3d)

# ========== DEBUG: 打印点云信息 ==========
print("\n=== DEBUG: Point Cloud Info ===")
pcd_points_original = np.asarray(pcd_cam.points)
print(f"Original point cloud (camera coord):")
print(f"  - Number of points: {len(pcd_points_original)}")
print(f"  - Min: {pcd_points_original.min(axis=0)}")
print(f"  - Max: {pcd_points_original.max(axis=0)}")
print(f"  - Mean: {pcd_points_original.mean(axis=0)}")

# 应用 cam_to_base
pcd_cam.transform(cam_to_base)
pcd_points_base = np.asarray(pcd_cam.points)
print(f"\nAfter cam_to_base:")
print(f"  - Min: {pcd_points_base.min(axis=0)}")
print(f"  - Max: {pcd_points_base.max(axis=0)}")
print(f"  - Mean: {pcd_points_base.mean(axis=0)}")

# 应用 ROBOT_PREDEFINED_TRANSFORMATION
pcd_cam.transform(ROBOT_PREDEFINED_TRANSFORMATION)
pcd_points_final = np.asarray(pcd_cam.points)
print(f"\nAfter ROBOT_PREDEFINED_TRANSFORMATION:")
print(f"  - Min: {pcd_points_final.min(axis=0)}")
print(f"  - Max: {pcd_points_final.max(axis=0)}")
print(f"  - Mean: {pcd_points_final.mean(axis=0)}")

# ========== 5. K3D 可视化 ==========
plot = k3d.plot()

# 5.1 渲染机械臂
print("\n=== DEBUG: Robot Mesh Transforms ===")
mesh_count = 0
for link, transform in cur_transforms.items():
    if link not in visuals_map: continue
    
    for v in visuals_map[link]:
        if v.geom_param is None: continue
        
        mesh_path = os.path.join("airexo/urdf_models/robot", v.geom_param)
        if not os.path.exists(mesh_path):
            continue

        mesh = trimesh.load(mesh_path, force='mesh')
        
        # 机械臂变换
        tf = ROBOT_PREDEFINED_TRANSFORMATION @ transform.matrix() @ v.offset.matrix()
        
        # DEBUG: 打印第一个 mesh 的变换
        if mesh_count == 0:
            print(f"First mesh: {v.geom_param}")
            print(f"Transform:\n{tf}")
            print(f"Mesh bounds before transform: {mesh.bounds}")
        
        mesh.apply_transform(tf)
        
        if mesh_count == 0:
            print(f"Mesh bounds after transform: {mesh.bounds}")
        
        mesh_count += 1
        
        plot += k3d.mesh(mesh.vertices.astype(np.float32), 
                         mesh.faces.astype(np.uint32),
                         color=0xaaaaaa)

print(f"Total meshes rendered: {mesh_count}")

# 5.2 渲染点云
pcd_colors = np.asarray(pcd_cam.colors).astype(np.float32)
pcd_colors_uint32 = (pcd_colors * 255).astype(np.uint32)
pcd_colors_packed = (pcd_colors_uint32[:, 0] << 16) | \
                    (pcd_colors_uint32[:, 1] << 8) | \
                     pcd_colors_uint32[:, 2]

plot += k3d.points(pcd_points_final, 
                   colors=pcd_colors_packed,
                   point_size=0.002,
                   shader='flat')

# 5.3 添加坐标轴
axis_size = 0.3
axes = k3d.vectors(
    origins=[[0, 0, 0], [0, 0, 0], [0, 0, 0]],
    vectors=[[axis_size, 0, 0], [0, axis_size, 0], [0, 0, axis_size]],
    colors=[0xff0000, 0x00ff00, 0x0000ff],
    line_width=0.01
)
plot += axes

print("\n=== Rendering ===")
plot.display()

Using camera: 105422061350

=== DEBUG: Calibration Info ===
Camera to base transformation:
[[-0.9994899   0.02875148  0.0139032   0.02070047]
 [-0.02373779 -0.3775733  -0.9256754   0.5817414 ]
 [-0.02136506 -0.9255333   0.3780632   0.699576  ]
 [ 0.          0.          0.          1.        ]]

ROBOT_PREDEFINED_TRANSFORMATION:
[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]]

Intrinsic matrix:
[[912.4466    0.      633.4127 ]
 [  0.      911.4704  364.21265]
 [  0.        0.        1.     ]]

=== DEBUG: Robot Transform (first link) ===
First link: base
Transform matrix:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]

=== DEBUG: Point Cloud Info ===
Original point cloud (camera coord):
  - Number of points: 864154
  - Min: [-1.15789476 -0.76121516  0.71600002]
  - Max: [1.0751782  0.76249442 2.52699995]
  - Mean: [ 0.04448005 -0.09284253  1.55684769]

After cam_to_base:
  - Min: [-1.02543512 -1.62514202  0.70226343]
  - Max: [ 1.20810909 -0.13443701  2.1628560

/home/haoxiang/miniforge3/envs/sam3/lib/python3.12/site-packages/traittypes/traittypes.py:98: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()